# IFRS17 シミュレーション 解約予実差


* 全てを実行するには、**Cell** メニューから**Run All**を選択します。


## このNotebookについて

``ifrs17sim`` モデルを作成し、選択した1つのモデルポイントについて、IFRSベースでのプロジェクションを行い、
IFRS17に関する諸数値と次のWaterfallチャートを描写します。

- PV Expected Cashflow Rollforward
- CSM Amortizatoin
- Actual Cashflow RollForwrd
- IFRS17 Financial Performance

ベース解約率に対して、乗数をかけて解約率前提を変更することができます。実績と予定に異なる乗数を乗じることにより、解約予実差がある場合のシミュレーションができます。

簡便的なモデルで、特に次の点が大きな簡略化・未対応点です。

- キャッシュフローのタイミングは期始か期末です。
- 事業費は全て保険キャッシュフローに含めています。
- 損失要素はモデル化されておらず、CSMが負を取りえるロジックになっています。
- カバー単位は保有保険金額に設定しています。
- 新契約費の償却率は契約始に固定しています。
- TVOGは無いとしています。
- リスク調整は実装されていません。




## 初期設定、モデルの作成

* 参照するモジュールをインポートします。
* ``ifrs17sim``モデルを作成し、``model``変数に代入します。次に作成するときは、``load_saved``に``True``を設定することによりモデル作成の時間を短縮できます。

In [ ]:
%matplotlib notebook

from draw_charts import (draw_waterfall, 
                         get_waterfalldata,
                         draw_actest_pairs)
import ifrs17sim

model = ifrs17sim.build(load_saved=True)

## モデルポイントの選択

* ``policy_id``に選択するモデルポイントのIDを代入します。
* Actualのプロジェクションを、``proj``、Estimatedのプロジェクションを``inner``に代入します。
* ``inner``は、Actualの時点をインデックスとしてもちます。例えば、``inner[0]``は、t=0時点における期待プロジェクション、``inner[1]``はt=1時点における期待プロジェクション、という具合です。

**演習**
* ``policy_id``を変更し、商品や契約属性が異なることにより、後のテーブルやグラフが変化することを確かめる。

In [ ]:
policy_id = 1
proj = model.OuterProj[policy_id]
inner = proj.InnerProj

## 解約率の変更

ベース解約率に対して乗じる`SurrRateMult`を、実績予定別、年度別に指定します。`[t, t+1]`間の解約率を指定して、その後の解約率を指定しないと`t+1`以降もその解約率が適用されます。


* `proj.SurrRateMult[t]` : `[t, t+1]`間の実績解約率
* `inner[t0].SurrRateMult[t1]` : `t0`時点の予定プロジェクションにおける、`[t1, t1+1]`間の予定解約率

**演習**

* 乗数を変更し、この後のグラフが変化することを確かめる。
* 変化した出力結果の妥当性を分析する。


In [ ]:
proj.SurrRateMult[1] = 1
proj.SurrRateMult[2] = 1
proj.SurrRateMult[3] = 1    # t=4以降もこの乗数が適用される

inner[1].SurrRateMult[1] = 1
inner[2].SurrRateMult[2] = 1
inner[3].SurrRateMult[3] = 1    # t=4以降もこの乗数が適用される

## 契約件数、解約率乗数、解約件数のグラフ

各時点において、実績解約率と予定解約率と、契約数ががどう推移するかを描写するグラフです。
* 最上段のグラフが`t=0`、その一段下のグラフが`t=1`というように、時間の推移に対応します。
* 線グラフは、各時点における、契約件数(`PolsIF_End`)、解約率乗数(`SurrRateMult`)、解約件数(`PolsSurr`)で、実線が実績、点線が予定です。
* 引数を変えることにより、出力する変数や期間を変更することも可能です。

**演習**

* グラフで表示する期間(実績、予定)を変えてみる。
* 他の変数のグラフを表示する。

In [ ]:
draw_actest_pairs(proj, inner, ['PolsIF_End', 'SurrRateMult', 'PolsSurr'], 5, 5)

## PV Expected Cashflow Rollforward

* 一つ目のセルは、各時点の期始の予定の将来キャッシュフロー現価と、現価をその期間でアンワインドした各項目のテーブルデータ(`estcf`)です。
* 二番目のセルは、そのテーブルのデータをWaterfallチャートで表示しています。
* 予実差がある場合、期末時点の実績保有契約数が、その期始時点で予定していた期末保有数と異なるので、転がし計算の結果と差異が生じます。

**演習**
* estcf (DataFrameオブジェクト) の各行の合計が、次の行の`PV_FutureCF`と一致するかどうか確かめる。
* 解約率乗数を変更して、残高や増減項目がどう変化するか確かめる。

In [ ]:
estcf = get_waterfalldata(
            proj,
            items=['PV_FutureCF',
                   'EstPremIncome',
                   'EstIntOnCF',
                   'EstAcqCashflow',
                   'EstClaim',
                   'EstExps'],
            length=3,
            reverseitems=['EstPremIncome'])
estcf

In [ ]:
draw_waterfall(estcf, title='PV Expected Cashflows')

## CSM Amortization

各時点のCSM残高と、CSMの増減項目からなるWaterfal chart用のデータと、グラフです。

**演習**

* 解約率乗数を変更して、CSMの増減項目がどう変化するか確かめる。

In [ ]:
csmrf = get_waterfalldata(
            proj, 
            items=['CSM_Unfloored',
                   'IntAccrCSM',
                   'AdjCSM_FlufCF',
                   'TransServices'],
            length=5,
            reverseitems=['TransServices'])
csmrf

In [ ]:
draw_waterfall(csmrf, title='CSM Amortization')

## Actual Cashflow RollForwrd

実績キャッシュフローのWaterfal chartです。累積キャッシュフローは全て現金として保有していると仮定しています。

In [ ]:
actcf = get_waterfalldata(
            proj,
            items=['PremIncome',
                   'IntAccumCF',
                   'ExpsAcqTotal',
                   'BenefitTotal',
                   'ExpsMaintTotal',
                   'ActualNetCF'],
            length=3,
            reverseitems=['ExpsAcqTotal',
                          'BenefitTotal',
                          'ExpsMaintTotal'])
actcf

In [ ]:
draw_waterfall(actcf, stocks=[0, 5], title='Actual Cashflows')

## IFRS17 Financial Performance

IFRSのPL/BSからなるWaterfal chartです。

**演習**

* 期待キャッシュフロー、CSM、実績キャッシュフローとどのような関係にあるか考察する。
* 対象契約や、出力期間を変更して出力する。
* 解約率乗数を変更して、CSMの増減項目がどう変化するか確かめる。

In [ ]:
ifrspl = get_waterfalldata(
            proj,
            items=['InsurRevenue',
                   'InsurServiceExps',
                   'InsurFinIncomeExps',
                   'ProfitBefTax'],
            length=5,
            reverseitems=['InsurServiceExps'])
ifrspl

In [ ]:
draw_waterfall(ifrspl, stocks=[0, 3], title='IFRS17 Profit/Loss')